In [1]:
import torch
import torch.nn as nn
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
import numpy as np
from gap_statistic import OptimalK
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score

import numpy as np
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import davies_bouldin_score
import os
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import plotly.io as pio
import numpy as np
import pandas as pd

pio.templates.default = "plotly_white"
os.environ["OMP_NUM_THREADS"] = '2'
df = pd.read_excel("data4.xlsx", header=0)
df["Date"] = pd.to_datetime(df["Date"])
df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 1,Unnamed: 2,Date,Contest number,Word,Number of reported results,Number in hard mode,1 try,...,6 tries,7 or more tries (X),ratio,vowel,len,uniq,star,three_s,three_ss,three_sss
0,0,0,NaN,NaN,2022-01-07,202,slump,-0.117038,1362,0.684682,...,-0.416685,-0.438101,0.016892,-1.278686,5,0.604574,-0.517873,0.27,27,0.030310
1,1,1,NaN,NaN,2022-01-08,203,crank,0.115919,1763,0.684682,...,0.390756,-0.196601,0.017369,-1.278686,5,0.604574,-0.517873,0.29,29,0.531068
2,2,2,NaN,NaN,2022-01-09,204,gorge,0.004022,1913,0.684682,...,1.682660,0.286398,0.020912,0.348316,5,-1.420179,-1.410324,0.17,17,-1.258570
3,3,3,NaN,NaN,2022-01-10,205,query,0.178765,2242,0.684682,...,0.875220,-0.196601,0.020927,0.348316,5,0.604574,-0.517873,0.21,21,-0.621527
4,4,4,NaN,NaN,2022-01-11,206,drink,0.700481,3017,0.684682,...,-1.062637,-0.438101,0.019606,-1.278686,5,0.604574,1.267029,0.45,45,3.079240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,350,350,NaN,NaN,2022-12-27,556,condo,-0.783899,2012,-0.594592,...,0.390756,0.044899,0.096365,0.348316,5,-1.420179,-1.410324,0.19,19,-2.272671
351,351,351,NaN,NaN,2022-12-28,557,impel,-0.791923,1937,-0.594592,...,-0.416685,-0.438101,0.096081,0.348316,5,0.604574,-0.517873,0.24,24,-1.506740
352,352,352,NaN,NaN,2022-12-29,558,havoc,-0.793698,1919,-0.594592,...,0.067779,-0.196601,0.095945,0.348316,5,0.604574,0.374578,0.18,18,-2.401559
353,353,353,NaN,NaN,2022-12-30,559,molar,-0.780272,1973,-0.594592,...,-0.416685,-0.438101,0.093048,0.348316,5,0.604574,-0.517873,0.25,25,-1.256361


In [2]:
df.iloc[:, 9:16]

,1 try,2 tries,3 tries,4 tries,5 tries,6 tries,7 or more tries (X)
0,0.684682,-0.696124,0.041752,1.124468,0.053344,-0.416685,-0.438101
1,0.684682,-0.195366,0.041752,-0.367817,0.053344,0.390756,-0.196601
2,0.684682,-0.696124,-1.247128,-1.113959,1.067829,1.682660,0.286398
3,0.684682,-0.445745,-0.860464,-0.554352,1.067829,0.875220,-0.196601
4,0.684682,0.806150,1.588408,0.191790,-1.299303,-1.062637,-0.438101
...,...,...,...,...,...,...,...
350,-0.594592,-0.946503,-0.731576,0.378326,0.898748,0.390756,0.044899
351,-0.594592,-0.696124,-0.216024,1.311004,0.222425,-0.416685,-0.438101
352,-0.594592,-0.946503,-0.860464,0.937933,1.067829,0.067779,-0.196601
353,-0.594592,-0.445745,-0.216024,0.937933,0.391506,-0.416685,-0.438101


In [3]:
df["three_ss"] = (df["three_ss"] - df["three_ss"].mean()) / df["three_ss"].std()

In [4]:
# data = pd.concat((df.iloc[:, 9:16], df["uniq"], df["star"]), axis=1)
data = pd.concat((df["three_ss"], df["uniq"], df["star"]), axis=1)
data = data.astype(float)
data = np.array(data)
data

array([[-0.16764407,  0.60457439, -0.51787291],
       [ 0.00688275,  0.60457439, -0.51787291],
       [-1.04027816, -1.42017945, -1.4103238 ],
       ...,
       [-0.95301475,  0.60457439,  0.37457798],
       [-0.34217089,  0.60457439, -0.51787291],
       [-0.86575134,  0.60457439, -0.51787291]])

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import LocallyLinearEmbedding
lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10)
pca = PCA(n_components=2)
# data = pca.fit_transform(data)

data = lle.fit_transform(data)
data

array([[-1.77153584e-04,  3.16160162e-02],
       [-1.74606325e-04, -6.68197636e-02],
       [ 1.02189312e-01,  4.07624280e-02],
       [-1.76440257e-04,  4.04971464e-03],
       [-1.77671527e-04,  5.16261843e-02],
       [ 9.89055101e-02,  1.51025888e-02],
       [-1.75284701e-04, -4.06154119e-02],
       [-1.73913142e-04, -9.36119949e-02],
       [-1.78030287e-04,  6.54954675e-02],
       [-1.78495556e-04,  8.34753452e-02],
       [-1.74820395e-04, -5.85505038e-02],
       [-1.73875298e-04, -9.50756389e-02],
       [ 1.00035236e-01,  6.57253850e-02],
       [-1.76401949e-04,  2.57046102e-03],
       [-1.77975596e-04,  6.33811653e-02],
       [-1.75460548e-04, -3.38203189e-02],
       [ 9.89053350e-02,  2.18665707e-02],
       [-1.75353511e-04, -3.79498232e-02],
       [-1.76898071e-04,  2.17420000e-02],
       [-1.75109609e-04, -4.73755434e-02],
       [-1.77272722e-04,  3.62096064e-02],
       [-1.75264495e-04, -4.13908021e-02],
       [-1.74121399e-04, -8.55600262e-02],
       [-1.

In [10]:
px.scatter(x=data[:, 0], y=data[:, 1])

In [7]:
# 使用 OptimalK 类进行 Gap Statistic 计算
optimalK = OptimalK(n_jobs=4, parallel_backend='multiprocessing')

n_clusters = optimalK(data, cluster_array=np.arange(2, 10))

n_clusters

8

In [8]:
gap = optimalK.gap_df
gap

,n_clusters,gap_value,ref_dispersion_std,sk,gap*,sk*,diff,diff*
0,2.0,6.464382,0.004263,0.024302,0.200807,0.231924,0.633807,0.218025
1,3.0,6.190242,0.007937,0.061059,0.152749,0.176617,-0.875756,0.163399
2,4.0,5.870692,0.003877,0.040117,0.110795,0.128014,NaN,NaN
3,5.0,6.786198,0.004397,0.058621,0.085268,0.098590,0.346118,0.116386
4,6.0,7.092802,0.001594,0.026804,0.068705,0.079356,1.865154,0.078203
5,7.0,5.275678,0.002582,0.048030,0.060985,0.070483,-1.761397,0.069958
6,8.0,7.169807,0.006413,0.132732,0.055232,0.064205,0.174241,0.062668
7,9.0,7.033167,0.001546,0.037601,0.047878,0.055314,0.305590,0.061200


In [9]:
for i in range(2, 10):
    kmeans = KMeans(n_clusters=i, max_iter=500)
    train_clusters = kmeans.fit_predict(data)
    si_score = metrics.silhouette_score(data, kmeans.labels_)
    ch_score = metrics.calinski_harabasz_score(data, kmeans.labels_)
    db_score = davies_bouldin_score(data, kmeans.labels_)
    print(i)
    print(si_score)
    print(ch_score)
    print(db_score)
    print("---------")

D:\Mysoftware_setup\anaconda3\envs\mypytorch\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

D:\Mysoftware_setup\anaconda3\envs\mypytorch\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.

D:\Mysoftware_setup\anaconda3\envs\mypytorch\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

D:\Mysoftware_setup\anaconda3\envs\mypytorch\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You c

2
0.9954782129749385
1126484.9457063815
0.011281298635299116
---------
3
0.9149287112454314
1837395.1166430698
0.29706226512671613
---------
4
0.9030690080196564
2046833.2030404573
0.35952977130775166
---------


D:\Mysoftware_setup\anaconda3\envs\mypytorch\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

D:\Mysoftware_setup\anaconda3\envs\mypytorch\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.

D:\Mysoftware_setup\anaconda3\envs\mypytorch\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

D:\Mysoftware_setup\anaconda3\envs\mypytorch\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You c

5
0.9109676192619883
2714050.3554771338
0.30130455988244015
---------
6
0.929511026921206
3939129.391843789
0.32160130018059657
---------


D:\Mysoftware_setup\anaconda3\envs\mypytorch\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

D:\Mysoftware_setup\anaconda3\envs\mypytorch\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.

D:\Mysoftware_setup\anaconda3\envs\mypytorch\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

D:\Mysoftware_setup\anaconda3\envs\mypytorch\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You c

7
0.9330977985198429
6109260.0513836555
0.3283980554779352
---------
8
0.9316795924450593
6653849.288935923
0.31462767120296475
---------


D:\Mysoftware_setup\anaconda3\envs\mypytorch\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

D:\Mysoftware_setup\anaconda3\envs\mypytorch\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.

D:\Mysoftware_setup\anaconda3\envs\mypytorch\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

D:\Mysoftware_setup\anaconda3\envs\mypytorch\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You c

9
0.9318914177827194
7857003.209928592
0.24624475110753016
---------
